In [1]:
from tqdm import tqdm,tqdm_notebook
from PIL import Image
import cv2
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
test_df = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')

In [3]:
def crop_image(img,threshold=10):
    '''
    return cropped image
    inspired by: https://codereview.stackexchange.com/questions/132914/crop-black-border-of-image-using-numpy
    
    '''
    img_grey=cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # convert image to greyscale
    mask = img_grey>threshold # find (black) pixels with greyscale value under the threshold 
    cropped_idx_arr=np.ix_(mask.any(1),mask.any(0))
    return img[cropped_idx_arr[0],cropped_idx_arr[1],:]

In [4]:
def smooth_image(img):
    '''
    apply smoothing 
    '''
    return cv2.addWeighted( img,2, cv2.GaussianBlur( img , (0,0) , 7) ,-2 ,128)

In [5]:
def resize_image(img, size):
    '''
    resize image to (size x size)
    '''
    return cv2.resize(img,(size, )*2)


In [6]:
def preprocess_image(image_path,size):
    img = cv2.imread(image_path)
    img = crop_image(img)
    img = resize_image(img, size)
    return smooth_image(img)

In [7]:
size=200

for i, image_id in enumerate(tqdm_notebook(train_df['id_code'])):
    image=preprocess_image(f'../input/aptos2019-blindness-detection/train_images/{image_id}.png',size=size)
    cv2.imwrite(f"{image_id}.png",image)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/3662 [00:00<?, ?it/s]

In [8]:
for i, image_id in enumerate(tqdm_notebook(test_df['id_code'])):
    image=preprocess_image(f'../input/aptos2019-blindness-detection/test_images/{image_id}.png',size=size)
    cv2.imwrite(f"{image_id}.png",image)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/1928 [00:00<?, ?it/s]

In [9]:
# !zip -m image.zip *.png